<h2> Tutorial - Regression Models with Keras</h2>

<h3>Objective for this Notebook<h3>    
<h5> 1. How to use the Tensorflow API, Keras, to build a regression model.</h5>
<h5> 2. Download and Clean dataset </h5>
<h5> 3. Build a Neural Network </h5>
<h5> 4. Train and Test the Network. </h5>     

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Download and Clean Dataset</a>  
2. <a href="#item32">Import Keras</a>  
3. <a href="#item33">Build a Neural Network</a>  
4. <a href="#item34">Train and Test the Network</a>  

</font>
</div>


<a id="item31"></a>


## Download and Clean Dataset


Let's start by importing the <em>pandas</em> and the Numpy libraries.


In [6]:
import pandas as pd
import numpy as np

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Let's download the data and read it into a <em>pandas</em> dataframe.


In [9]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 


#### Let's check how many data points we have.


In [12]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [15]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [16]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors and target


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [20]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>


Let's do a quick sanity check of the predictors and the target dataframes.


In [23]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [24]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.


In [26]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's save the number of predictors to *n_cols* since we will need this number when building our network.


In [28]:
n_cols = predictors_norm.shape[1] # number of predictors

<a id="item1"></a>


<a id='item32'></a>


## Import Keras


Recall from the videos that Keras normally runs on top of a low-level library such as TensorFlow. This means that to be able to use the Keras library, you will have to install TensorFlow first and when you import the Keras library, it will be explicitly displayed what backend was used to install the Keras library. In CC Labs, we used TensorFlow as the backend to install Keras, so it should clearly print that when we import Keras.


#### Let's go ahead and import the Keras library


In [34]:
from tensorflow import keras

As you can see, the TensorFlow backend was used to install the Keras library.


Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.


In [37]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input

<a id='item33'></a>


## Build a Neural Network


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [41]:
# define regression model
def regression_model(input_dim):
    # create model
    model = Sequential([
        Input(shape=(input_dim,)),  # Explicit input layer
        Dense(50, activation='relu'),                    # First hidden layer
        Dense(50, activation='relu'),                     # Second hidden layer
        Dense(1, activation='linear')                     # Output layer for regression
    ])

    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has two hidden layers, each of 50 hidden units.


<a id="item4"></a>


<a id='item34'></a>


## Train and Test the Network


Let's call the function now to create our model.


In [47]:
# build the model
model = regression_model(n_cols)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 50)             │           450 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 50)             │         2,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,051 (11.92 KB)

 Trainable params: 3,051 (11.92 KB)

 Non-trainable params: 0 (0.00 B)

Now we will split our data into training and testing with a 70%-30% split (test_size=0.3).   

70% of our data will be used for training :

X_train : Our training data containing the variables the model will use to learn how to predict.

y_train : Our training data containing the labels.

#
30% of our data will be used for testing its performance metrics :

X_test : Unseen data by the model, we will use it to make predictions with our trained model.

y_test : The labels associated with X_test data, we will compare them with the predictions that our model provided as output when shown X_test and that way we will obtain performance metrics.

In [49]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

Next, we will train the model using the *fit* method. We will train the model for 100 epochs.


In [51]:
# fit the model
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
23/23 - 0s - 11ms/step - loss: 1541.2819
Epoch 2/100
23/23 - 0s - 910us/step - loss: 1428.5844
Epoch 3/100
23/23 - 0s - 930us/step - loss: 1262.0283
Epoch 4/100
23/23 - 0s - 851us/step - loss: 1013.7786
Epoch 5/100
23/23 - 0s - 760us/step - loss: 699.7716
Epoch 6/100
23/23 - 0s - 747us/step - loss: 418.7799
Epoch 7/100
23/23 - 0s - 727us/step - loss: 263.0559
Epoch 8/100
23/23 - 0s - 728us/step - loss: 215.7027
Epoch 9/100
23/23 - 0s - 708us/step - loss: 199.4929
Epoch 10/100
23/23 - 0s - 703us/step - loss: 190.4789
Epoch 11/100
23/23 - 0s - 729us/step - loss: 182.9215
Epoch 12/100
23/23 - 0s - 772us/step - loss: 176.4402
Epoch 13/100
23/23 - 0s - 756us/step - loss: 171.6512
Epoch 14/100
23/23 - 0s - 731us/step - loss: 167.5122
Epoch 15/100
23/23 - 0s - 732us/step - loss: 163.7119
Epoch 16/100
23/23 - 0s - 741us/step - loss: 159.7008
Epoch 17/100
23/23 - 0s - 737us/step - loss: 156.7447
Epoch 18/100
23/23 - 0s - 739us/step - loss: 154.1825
Epoch 19/100
23/23 - 0s - 728us/st

### Now lets evaluate our model and get performance metrics !

First, since our model only saw the training data and learned from it, we will make predictions on the unseen test set.

y_pred are the predictions that our model generated from it's experience when shown the predictor variables of X_test.

That way we can compare y_pred (predictions made by our model) with y_test (the real labels of X_test) and see the difference as performance metrics.

In [53]:
# Make predictions on the test set
y_pred = model.predict(X_test)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


Since this is a regression model we will be using mean squared error, mean absolute error and the r squared score to evaluate it.

In [55]:
# Calculate and print regression metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Mean Absolute Error (MAE): {mae:.4f}')
print(f'R² Score: {r2:.4f}')

Mean Squared Error (MSE): 43.4408
Mean Absolute Error (MAE): 5.0672
R² Score: 0.8394


Feel free to vary the following and note what impact each change has on the model's performance:

1. Increase or decreate number of neurons in hidden layers
2. Add more hidden layers
3. Increase number of epochs


In [57]:
# define regression model
def regression_model_version2(input_dim):
    # create model
    model = Sequential([
        Input(shape=(input_dim,)),                         # Explicit input layer
        Dense(128, activation='relu'),                    # First hidden layer
        Dropout(0.2),                                     # Dropout for regularization
        Dense(64, activation='relu'),                     # Second hidden layer
        Dense(32, activation='relu'),                     # Third hidden layer
        Dropout(0.2),
        Dense(16, activation='relu'),                     # Fourth hidden layer
        Dense(1, activation='linear')                     # Output layer for regression
    ])
    
    # Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [58]:
model_version2 = regression_model_version2(input_dim=n_cols)  # Build the model
model_version2.summary()  # Print model summary

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 128)            │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,033 (47.00 KB)

 Trainable params: 12,033 (47.00 KB)

 Non-trainable params: 0 (0.00 B)

In [59]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

In [60]:
# fit the model
model_version2.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
23/23 - 0s - 18ms/step - loss: 1506.5284
Epoch 2/100
23/23 - 0s - 1ms/step - loss: 1012.5695
Epoch 3/100
23/23 - 0s - 1ms/step - loss: 361.9189
Epoch 4/100
23/23 - 0s - 1ms/step - loss: 248.3746
Epoch 5/100
23/23 - 0s - 811us/step - loss: 226.4765
Epoch 6/100
23/23 - 0s - 803us/step - loss: 211.5013
Epoch 7/100
23/23 - 0s - 793us/step - loss: 204.6122
Epoch 8/100
23/23 - 0s - 816us/step - loss: 212.2970
Epoch 9/100
23/23 - 0s - 844us/step - loss: 188.3201
Epoch 10/100
23/23 - 0s - 874us/step - loss: 183.7838
Epoch 11/100
23/23 - 0s - 1ms/step - loss: 172.6186
Epoch 12/100
23/23 - 0s - 1ms/step - loss: 166.8078
Epoch 13/100
23/23 - 0s - 953us/step - loss: 156.2429
Epoch 14/100
23/23 - 0s - 939us/step - loss: 163.5040
Epoch 15/100
23/23 - 0s - 926us/step - loss: 162.4820
Epoch 16/100
23/23 - 0s - 873us/step - loss: 139.1870
Epoch 17/100
23/23 - 0s - 850us/step - loss: 134.5186
Epoch 18/100
23/23 - 0s - 821us/step - loss: 135.7166
Epoch 19/100
23/23 - 0s - 833us/step - loss: 1

In [61]:
# Make predictions on the test set
y_pred = model_version2.predict(X_test)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [62]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Mean Absolute Error (MAE): {mae:.4f}')
print(f'R² Score: {r2:.4f}')

Mean Squared Error (MSE): 31.8443
Mean Absolute Error (MAE): 4.2934
R² Score: 0.8823


For other models, other types of performance metrics are more meaningfull. It all depends on context and the end goal of the model. 

Research, planning and analysing various metrics with a clearly defined end goal is key to understanding how capable a model is and what are his limitations.

Keep that in mind when tackling the next exercises!

### Thank you for following this tutorial - feel free to go back and play with the architcture and parameters